<center> <img src="https://upload.wikimedia.org/wikipedia/commons/c/c8/Iteso_logo.jpg" align="center" width="200" height="200"/> </center>

<center><font color=#555555><font size=6> Second Laboratory </font> <br> <br>
    
<center><font color=#555555><font size=5> Microstructures and Trading Systems </font> <br> <br> <br> <br>

<center><font color=#555555><font size=6> Bryan Manica Pineda</font> <br> <br>
    
<center><font color=#555555><font size=4> if722176@iteso.mx</font> <br> <br>
   
<center> <font color= #555555> <font size = 4> June. 2022 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/bmanica/aptmodel-lab2'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> High Frequency Models Testing </font> <br> <br> <font color= #555555> <font size = 5> APT and Roll Model into real world data  </font>

<hr style="border:0.02in solid gray"> </hr>

<font color= #555555> <font size = 4> Abstract </font>

<font color= #6B6B6B> <font size = 3>  Include here a description of the whole work, use no more than 400 words, you can use **Bold** and *Italic* highlights, also you can inclue [links](www.iteso.mx). Usually is a good strategy to generate this text at the very end of the process, since you have to include all aspects of the work, from a brief introduction, to problem definitions and methods used, to finally the results found. </font>

<br>

# <font color= #6B6B6B> <font size = 6> 1. Introduction </font>

<hr style="border:0.01in solid gray"> </hr>

This is an introduction, normally, it is done at the very end, when all the other content in the notebook is completed but just before the abstract (which is short version of the introduction) ... 

<br>

# <font color= #6B6B6B> 2. Install/Load Packages and Depedencies </font>

<hr style="border:0.01in solid gray"> </hr>

## <font color= #6B6B6B> 2.1 Python Packages </font> <font color= #555555>

Para poder correr el presente **notebook** es necesario haber instalado todas aquellas librerias que se encuentran descritas en el archivo adjunto en el repositorio con nombre _requirements.txt_. Dichas dependencias son las siguientes:

- **pandas** >= 1.1.0
- **numpy** >= 1.19.1
- **jupyter** >= 1.0.0
- **chart_studio** >= 1.1
- **plotly** >= 4.14
- **statsmodels** >= 0.13.2

## <font color= #6B6B6B> 2.2 Files Dependencies </font> <font color= #555555>

Además de todas las librerias previamente mencionadas, el correcto funcionamiento de este **notebook** también requiere de algunos datos externos. Estos archivos se encuentran directamente en la carpeta _files_ adjunta dentro de este mismo repositorio:

- files/orderbooks_05jul21.json: Archivo tipo json. Datos históricos de diversos orderbooks correspondientes a una hora el 5 de julio del 2021.
- files/btcusdt_binance.csv: Archivo tipo csv. Datos históricos respecto a los public trades de **BTCUSDT**.

## <font color= #6E6E6E> 2.3 Install Packages</font>

En caso de que no se hayan instalado los paquetes dentro del ambiente virtual sobre el cual se está trabajando, se pueden instalar directamente dentro de este cuaderno. A continuación se encuentra la estructura de comandos necesarias para instalar las librerias, solo correr en caso de que no se hayan instalado previamente:

In [1]:
%%capture

# Install all the pip packages in the requirements.txt
import sys
!{sys.executable} -m pip install -r requirements.txt

## <font color= #6E6E6E> 2.3 Load Packages and Scripts</font>

Ya con los requerimentos computacionales bien definidos, ahora se proseguirá a cargar tanto las librerias como los scripts de python 3 para poder realizar el análisis correspondiente.

In [1]:
### Required packages
import pandas as pd
import numpy as np
import warnings

### Required local scripts
import data as dt
import functions as fn
import visualizations as vz

<br>

# <font color= #6B6B6B> 3. Data Description </font>

<hr style="border:0.01in solid gray"> </hr>

Para poder realizar este laboratorio en particular, se nos fueron proporcionados los datos previamente mencionados. A continuación se comenzará a describir un poco más a detalle la estructura de los mismos para entender de mejor manera la materia prima sobre la cual se estará trabajando.

**Order Book.**

Este conjunto de datos se encuentra bajo el formato _json_, por tanto se compone por una lista de diccionarios donde cada subconjuto de valores responde a un libro de órdenes del activo con clave de pizarra **_BTCUSDT_**, dicho orderbook se encuentra asociado a cada punto del tiempo que representa la llave del diccionario. La actualización de dichos libros se da en términos de segundos, por lo tanto la ventana de tiempo sobre la cual trabajaremos es de un poco más de $1$ hora. A pesar de que pareciera que no tenemos tantos datos, la realidad es que cada libro cuenta con una cantidad suficiente de datos para poder extraer información relevante a ese punto del tiempo que se encuentra asociado.

**Public Trades.**

Este segundo y último conjunto de datos se encuentra en formato _csv_, y básicamente hace referencia a los movimientos de mercado (trades) que se realizan en cada punto del tiempo, nuevamente aquí estamos trabajando con aquellos datos donde hubo una conexión entre la parte compradora y la parte vendedora del mercado para el activo **_BTCUSDT_**, es decir, en estos nuevos datos contamos con la información de aquellas órdenes que sí fueron empatadas por lo tanto estas operaciones que se efectuan son las que al final del día terminan teniendo un impacto directo en la fluctuación del precio para el activo analizado. La ventana de tiempo para la cual disponemos de esta información se encuentra alrededor de $1$ día.

## <font color= #6B6B6B> 3.1. Order Book </font>

Contamos entonces con una lista de diccionarios donde cada llave representa un punto en el tiempo donde se contaba con ese determinado libro de órdenes, ahora lo que haremos será ir definiendo y mostrando un poco mas sobre los datos ya desde una perspectiva un poco más visual.

In [2]:
### Let's visualize the keys range.
ob_data = dt.ob_data_bit
range_ob = pd.to_datetime(pd.Series(list(ob_data.keys()))).sort_values()
print(f'El primer tiempo registrado está en: {range_ob[0]}, y el último en: {range_ob.tail(1).item()}')

El primer tiempo registrado está en: 2021-07-05 13:06:46.571000+00:00, y el último en: 2021-07-05 14:06:46.412000+00:00


Como podemos observar ya directamente de la descripción de los rangos para las llaves de los orderbooks contamos con un poco menos de $1$ hora de información correspondiente al día **2021-07-05** entre $1$ y $2$ de la tarde.

In [3]:
### Let's see how many books do we have
print(f'En una hora de tiempo contamos con {len(list(ob_data.keys())):,} libros')

En una hora de tiempo contamos con 2,401 libros


Entonces si bien pareciera que $1$ hora de tiempo no puede aportar mucha información, la realidad es que dentro de este horario contamos con alrededor de $2,401$ libros. A continuación se mostrará un poco respecto al contenido de cada orderbook, esto con la intención de alcanzar a percibir la cantidad de datos que tenemos.

In [4]:
### Let's see the first orderbook values
print(f'Este primer orderbook corresponde al punto en el tiempo. {range_ob[0]}:')
list(ob_data.values())[0].head(5)

Este primer orderbook corresponde al punto en el tiempo. 2021-07-05 13:06:46.571000+00:00:


,bid_size,bid,ask,ask_size
0,0.000400,28270.0,28275.0,0.025405
1,0.009787,28269.0,28276.0,0.516810
2,0.008168,28268.0,28277.0,0.005044
3,0.995787,28266.0,28278.0,0.377374
4,1.038704,28265.0,28280.0,1.179715


Como podemos percibir del dataframe correspondiente al primer punto del tiempo, contamos con diferentes atributos:

- **bid_size**: Corresponde al volumen posicionado al del **bid** asociado.
- **bid**: Responde directamente al precio más alto que un comprador esta dispuesto a pagar por el activo de interés.
- **ask**: Es el precio más bajo al cual un vendedor esta dispuesto a vender el activo.
- **ask_size**: Corresponde al volumen posicionado al del **ask** asociado.

Todas y cada una de las llaves correspondientes a los datos del orderbook contienen un dataframe con las mismas características que se encuentra desplegado. Entonces a partir de estos valores es que se estará experimentando y observando la eficacia de los modelos de alta frecuencia propuestos.

## <font color= #6B6B6B> 3.2. Public Trades </font>

Los trades públicos de cierta manera se encuentran contenidos en una estructura de datos mucho más amigable para la mayoría (_csv_), por tanto python se encarga de leer estos archivos y directamente transformarlos en un dataframe. A continuación observaremos un poco respecto a los atributos y dimensionas que trae consigo este archivo.

In [5]:
### Let's visualize the df range.
pt_data = dt.pt_data
range_pt = pd.to_datetime(pt_data.timestamp).sort_values()
print(f'El primer tiempo registrado está en: {range_pt.head(1).item()}, y el último en: {range_pt.tail(1).item()}')

El primer tiempo registrado está en: 2022-05-10 00:00:12, y el último en: 2022-05-10 23:59:38


Como podemos observar directamente de los rangos de tiempo para los public trades, vemos que el primer registro se efectuó en la madrugada del **2022-05-10** y el último de ellos se dió el mismo día justo antes de que termine, por lo tanto para este conjunto de información contamos con todo un día de transacciones.

In [6]:
### Let's see the public trades dataframe
print(f'Los public trades con los que contamos tienen un total de {pt_data.shape[0]:,} filas')
pt_data.head()

Los public trades con los que contamos tienen un total de 286,600 filas


,timestamp,price,amount,side
0,2022-05-10 08:08:07,31702.47,0.00035,sell
1,2022-05-10 08:08:07,31702.48,0.00263,buy
2,2022-05-10 08:08:07,31702.48,0.00631,buy
3,2022-05-10 08:08:07,31702.48,0.00264,buy
4,2022-05-10 08:08:07,31702.48,0.01146,buy


Del registro de transacciones para el **_BTCUSDT_** contamos con un total de $286,000$ operaciones, las cuales se encuentran dividas a lo largo de un día, en cuanto a los atributos del data frame contamos con el tiempo, el precio al cual se hizo la conexión, el volumen transaccionado a dicho precio en ese punto del tiempo, y finalmente contamos con la estructura de venta o compra lo cual indica de que lado se efectuó la operación.

<br>

# <font color= #6B6B6B> 4. APT Model Testing </font>

<hr style="border:0.01in solid gray"> </hr>

Con la intención de poder eficientar tiempos computacionales se decide trabajar sobre una serie de _scripts_ de **python 3**, los cuales se encuentran anexos al repositorio <font color= #555555> <a href='https://github.com/bmanica/aptmodel-lab2'>aptmodel-lab2</a></font>. En ellos se encuentra predefinida una serie de cálculos los cuales nos ayudarán a realizar el test y analizar el cumplimiento del modelo frente a situaciones de la vida real.

Antes de poder entrar en detalle respecto a la serie de pruebas realizadas para el **APT**, es importante tener claro el origen y la teoría dentrás de este modelo de alta frecuencia. En este sentido el modelo parte de su definición básica _"poder encontrar el pago futuro respecto a un activo financiero que presenta flujos"_, de manera que la ecuación básica para partir es la siguiente:

$$x_{t+1}=p_{t+1}+d_{t+1}$$

donde:

- $x_{t+1}: \text{Pago del activo a realizarse a un momento futuro}$
- $p_{t+1}: \text{Precio futuro de la acción}$
- $d_{t+1}: \text{Dividendo que se espera pague la acción a un momento futuro}$

Ahora hasta el momento se encuentra definido el pago que se podría esperar en un momento $t+1$, sin embargo el concepto económico que nos ayudará a modelar el impacto de ese pago futuro se dará a través de la **función de utilidad** de los consumos actuales y futuros de un inversionista:

$$U(c_{t}, c_{t+1})=U(c_{t})+\beta U(c_{t+1})$$

donde:

- $c_{t}: \text{Consumo en el tiempo presente}$
- $c_{t+1}: \text{Consumo con rezago a un tiempo futuro}$
- $\beta: \text{Factor de descuento subjetivo}$

Dentro de estas dotaciones de consumo para el inversionista tanto en tiempo presente $(t)$ como tiempo futuro $(t+1)$ existen ciertas restricciones que limitan su consumo, de manera que:

$$c_{t} = e_{t}-p_{t}K$$
$$c_{t+1} = e_{t+1}+x_{t+1}K$$

donde:

- $e_{t}: \text{Refiere al consumo presente}$
- $p_{t}: \text{Refiere al precio del activo riesgoso en tiempo presente}$
- $K: \text{Refiere al número de titulos del activo que el inversionista puede adquirir con limitantes de capital}$
- $e_{t+1}: \text{Refiere al consumo a un tiempo futuro}$
- $x_{t+1}: \text{Refiere al pago esperado a un tiempo futuro}$

En este sentido y con una definición más clara de la utilidad y como se modela o define para cada punto en el tiempo, la pregunta natural surge al decir, ¿qué puedo definir con eso?. Para ello es importante pensar en cuales son los incentivos al consumo de un agente económico, el más claro de ellos debería corresponder al alcanzar la satisfacción máxima con la cantidad de recursos mínimos necesarios, entonces podríamos comenzar a definir al modelo del **APT** como un problema de optimización (maximización) donde se busca obtener la mayor utilidad posible con un consumo de $K$ en el activo riesgoso.

$$max_{{K}} U(c_{t}) + E\big[\beta U(c_{t+1})\big]$$

Donde podemos comenzar a observar la presencia del operador de **esperanza matemática** $(E[x])$ el cual surge de la necesidad de análisis de un proceso estocástico no conocido $(x_{t+1})$ por lo tanto para esta componente la mejor manera de aproximar su valor se da a través del propio valor esperado asociado a su función de densidad de probabilidad.

Para poder encontrar este punto critico se define a través del críterio de primer orden, donde se deriva nuestra expresión con respecto a $K$ y ese valor se iguala a $0$. Después de realizar dicho proceso se encuentra que:

$$p_{t} = E \bigg[\beta \frac{U'(c_{t+1}}{U'(c_{t})} x_{t+1} \bigg] \longrightarrow p_{t} = E [m_{t+1} x_{t+1}]$$

donde:

- $m_{t+1}: \text{Hace referencia al factor estocástico de descuento parametrizado a la utilidad de consumo del inversionista}$

Partiendo entonces de la definición de $x_{t+1} = p_{t+1}+d_{t+1}$ y retomando la ecuación del precio $p_{t}$ es a este punto donde los **principales supuestos** del modelo se empiezan a definir. Es importante recordar que el enfoque sobre el cual se define el **APT** en este contexto es sobre la microestructura del mercado, la cual maneja diferenciales de tiempo muy pequeños (en mercados lo suficientemente líquidos), es por ello que se pueden definir y dar congruencia a los siguientes supuestos:

- $d_{t+1} = 0: \text{En intervalos de tiempo muy pequeños no se puede esperar el flujo otorgado por los dividendos}$
- $\beta \approx 1: \text{No existe preferencia de consumo a largo plazo}$
- $U'(c_{t+1}) = U'(c_{t}): \text{No se afecta la incertidumbre de consumo}$

Con la aplicación de estos supuestos finalmente obtenemos el resultado final para el precio en tiempo $t+1$

$$E \big[p_{t+1} \big] = p_t$$

Es decir, la mejor estimación del precio que se puede esperar a un punto futuro en el tiempo (según el **APT**) es el precio actual. Este tipo de proceso estocástico es muy particular y se le conoce como **martingala**

En este sentido el **APT** nos menciona que ante la incertidumbre que pueda llegar a existir en periodos futuros de tiempo la zona segura siempre será la cotización más actual (si y solo si para periodos de tiempo muy pequeños). Con la intención de comprobar la eficiencia del modelo frente a una situación real de mercado se estará contrastando a través de los siguientes experimentos:

- **Experimento 1:** Número de ocurrencias en el que $p_{t+1} = p_{t}$

- **Experimento 2:** Número de ocurrencias en el que $t_{t+1} \neq p_{t}$

Y es a través de estos experimentos y pruebas que se definirá una proporción que nos ayude a entender y reflejar que tanto se cumple la definción formal del **APT** en una microestructura de mercado tan volátil como lo es la de _Bitcoin_. Es importante mencionar que para términos de prueba se definirá a $p_t$ a través de dos grandes maneras de calcular el precio de un activo, el **Simple Mid-Price** y el **Weighted Mid-Price**.

In [7]:
### Let's define the apt testing data frames.
apt_all = fn.apt_check_all(ob_data)
apt_tob = fn.apt_check_tob(ob_data)

print(f'The keys for apt model test with all orders are: {list(apt_all.keys())}')
print(f'The keys for apt model test with tob orders are: {list(apt_tob.keys())}')

The keys for apt model test with all orders are: ['simple_mid_price', 'weighted_mid_price']
The keys for apt model test with tob orders are: ['simple_mid_price', 'weighted_mid_price_a', 'weighted_mid_price_b']


## <font color= #6B6B6B> 4.1. Testing All Orders </font>

Este primer enfoque que se propone analizar incluye todas las órdenes de todos los libros de órdenes presentes en el diccionario de entrada, es decir, para este primer punto de análisis queremos observar los cambios en los precios de prácticamente todas las potenciales operaciones de los orderbooks (aproximadamente unos $60,000$ registros a consolidar). La metodología que se propone es la misma que se menciona en la definición del modelo, se busca encontrar la cantidad de veces que el **Mid-Price** sigue un proceso estocástico tipo _Martingala_ para cada minuto en el tiempo registrado. 

### <font color= #6B6B6B> 4.1.1. Simple Mid-Price </font>

Para este análisis se calcula es **Simple Mid-Price** el cual se define matemáticamente por la siguiente expresión:

$$\text{Mid-Price} = \frac{\text{bid} + \text{ask}}{2}$$

Y es este valor el que se calcula para todas y cada una de las órdenes presentes en cada orderbook del rango de datos con el que se cuenta.

In [8]:
### Let's see the data frame for simple mid-price test
print(f'''The min proportion where the mid-price follows martingale process is {apt_all['simple_mid_price']['P. Exp 1'].min()}
and it happened on {apt_all['simple_mid_price']['P. Exp 1'].idxmin()}''')
print("")
print(f'''The mean proportion where the mid-price follows martingale process is:
{round(apt_all['simple_mid_price']['P. Exp 1'].mean(),4)*100}%''')

apt_all['simple_mid_price'].head()

The min proportion where the mid-price follows martingale process is 0.67
and it happened on 2021-07-05 13:31:00

The mean proportion where the mid-price follows martingale process is:
80.41%


,Exp 1,Exp 2,P. Exp 1,P. Exp 2
Time,,,,
2021-07-05 13:06:00,177,47,0.79,0.21
2021-07-05 13:07:00,781,243,0.76,0.24
2021-07-05 13:08:00,799,200,0.80,0.20
2021-07-05 13:09:00,750,224,0.77,0.23
2021-07-05 13:10:00,789,235,0.77,0.23


Para esta primera estructura del experimento con **Simple Mid-Price** de todas las órdenes presentes podemos observar directamente del data frame resultante que en general las proporciones donde el precio sigue un proceso tipo **martingala** son relativamente altas, con un $80\%$ de las veces en promedio y con un valor mínimo que ronda en el $67\%$ en la consolidación por minutos. 

Este resultado nos da un primer acercamiento al modelo y a su practicidad en la aplicación del mismo, sin embargo a falta de más pruebas no se puede concluir nada aún.

### <font color= #6B6B6B> 4.1.2. Weighted Mid-Price </font>

Para términos del **Weighted Mid-Price** se calculó a través de la siguiente expresión matemática:

$$\text{Weighted Mid-Price} = \bigg(\frac{\text{bv}}{\sum_{i=1}^{n}\big(\text{bv}_{i}+\text{av}_{i}\big)} \bigg) \text{ap} + \bigg(\frac{\text{av}}{\sum_{i=1}^{n}\big(\text{bv}_{i}+\text{av}_{i}\big)} \bigg) \text{bp}$$

donde:

- $\text{bv}: \text{bid volume}$
- $\text{av}: \text{ask volume}$
- $\text{ap}: \text{ask price}$
- $\text{bp}: \text{bid price}$

En este sentido y al estar tomando practicamente todos las órdenes presentes, la profundidad detonada por $i$ solo se define para su mismo nivel.

In [9]:
### Let's see the data frame for weighted mid-price test
print(f'''The max proportion where the mid-price follows martingale process is {apt_all['weighted_mid_price']['P. Exp 1'].max()}
and it happened on {apt_all['weighted_mid_price']['P. Exp 1'].idxmax()}''')
print("")
print(f'''The mean proportion where the mid-price follows martingale process is:
{round(apt_all['weighted_mid_price']['P. Exp 1'].mean(),4)*100}%''')

apt_all['weighted_mid_price'].head()

The max proportion where the mid-price follows martingale process is 0.01
and it happened on 2021-07-05 14:02:00

The mean proportion where the mid-price follows martingale process is:
0.02%


,Exp 1,Exp 2,P. Exp 1,P. Exp 2
Time,,,,
2021-07-05 13:06:00,0,224,0.0,1.0
2021-07-05 13:07:00,0,1024,0.0,1.0
2021-07-05 13:08:00,0,999,0.0,1.0
2021-07-05 13:09:00,0,974,0.0,1.0
2021-07-05 13:10:00,0,1024,0.0,1.0


Ahora para el mismo enfoque pero definiendo a $p_t$ como el **Weighted Mid-Price** a primera instancia se logra percibir un cambio radical en los resultados obtenidos previamente para el **Simple Mid-Price**. La proporción máxima en cada punto del tiempo se encuentra alrededor del $1\%$ de ocasiones donde el precio sigue un proceso tipo **martingala** un reducción considerable del $66\%$ respecto al **Simple Mid-Price** probado anteriormente. Además la proporción promedio se ve reducida a un $0.02\%$ un valor muy cercano a $0$ que nos indicaría que el modelo del **APT** tiene muy poca efectividad en la estructura de datos probada.

En este sentido surge una pregunta natural, ¿por qué se visualiza un cambio tan drástico?. La realidad es que a pesar de que se esta manejando la misma estructura de datos en ambos casos e incluso a pesar de que el **Weighted Mid-Price** arroja un valor no tan alejado del **Simple Mid-Price**, la clave para entender este comportamiento está en el _orderbook_. Si recordamos cada libro de órdenes tiene su profundidad, y a medida que vamos "bajando" en dichos niveles podemos encontrar una mayor disparidad en la voluntades de compra y venta de los inversionistas, ahora esas voluntades no solo cambian de precio a precio, también lo hacen a nivel volumen y es justamente aquí donde se encuentra la principal razón de cambio, ya que el **Simple Mid-Price** al ser un promedio sencillo pondera de igual manera las órdenes de compra y de venta para definir el precio, mientras que el **Weighted Mid-Price** sí hace una diferenciación en este sentido al ser un promedio ponderado.

## <font color= #6B6B6B> 4.2. Testing Top of the Book Orders </font>

En este segundo enfoque de análisis se busca realizar un proceso análogo al previamente descrito, sin embargo, la principal diferencia para este punto se da en que ahora solo se utilizarán los **Mid-Prices** que se definen única y exclusivamente para el _Top of the Book_, es decir, esta nueva propuesta solo tomará un precio para cada libro de órdenes, de manera que el procesamiento de datos se reducirá considerablemente ya que ahora solo se contarán con $2,400$ registros a consolidar por minuto.

### <font color= #6B6B6B> 4.2.1. Simple Mid-Price </font>

Para el cálculo de este **Simple Mid-Price** se define prácticamente la misma fórmula que se utiliza para todas las órdenes, la diferencia principal radica en que ahora solo se tomará el _Top of the Book_, en otras palabras este nuevo análisis solo tomará el mejor **bid** y el mejor **ask** presente en cada libro de órdenes.

En este sentido y tratando de llegar a un análisis comparativo entre los **Mid-Prices** del _Top of the Book_ y el resto de las órdenes, podríamos comenzar a intuir que las proporciones donde se cumpla que el precio sigue una martingala con esta nueva estructura de análisis será menor, ya que al tomar siempre el _Top of the Book_ podríamos decir que estaríamos más cerca de "definir" la tendencia del precio a cada punto del tiempo, y en ese sentido se podría esperar que las fluctuaciones en el precio fueran mayores en comparación a si tomamos todas las órdenes presentes.

In [10]:
### Now let's render the simple mid-price experiment just for tob
print(f'''The min proportion where the mid-price follows martingale process is {apt_tob['simple_mid_price']['P. Exp 1'].min()}
and it happened on {apt_tob['simple_mid_price']['P. Exp 1'].idxmin()}''')
print("")
print(f'''The mean proportion where the mid-price follows martingale process is:
{round(apt_tob['simple_mid_price']['P. Exp 1'].mean(),4)*100}%''')

apt_tob['simple_mid_price'].head()

The min proportion where the mid-price follows martingale process is 0.65
and it happened on 2021-07-05 13:52:00

The mean proportion where the mid-price follows martingale process is:
72.39%


,Exp 1,Exp 2,P. Exp 1,P. Exp 2
Time,,,,
2021-07-05 13:06:00,6,3,0.67,0.33
2021-07-05 13:07:00,27,14,0.66,0.34
2021-07-05 13:08:00,31,9,0.78,0.22
2021-07-05 13:09:00,27,12,0.69,0.31
2021-07-05 13:10:00,30,11,0.73,0.27


In [11]:
### Let's see the mean proportion for experiment 1.
print(f'''With all orders we have a mean proportion of {round(apt_all['simple_mid_price']['P. Exp 1'].mean(),4)*100}%''')
print(f'''With tob orders we have a mean proportion of {round(apt_tob['simple_mid_price']['P. Exp 1'].mean(),4)*100}%''')

With all orders we have a mean proportion of 80.41%
With tob orders we have a mean proportion of 72.39%


Para este segundo enfoque de los experimentos realizados, el cambio principal como se comentó radica en que ahora solo se tomarán órdenes del _top of the book_, las cuales tienen la característica de representar el "mejor" **Bid** y el "mejor" **Ask** de cada libro de órdenes. En este sentido se esperaría que exista una diferencia de precios un poco más notoria de libro a libro, situación que se ve reflejada directamente en los experimentos realizados, donde ahora la proporción mínima donde el precio sigue un proceso tipo **martingala** se encuentra en $65\%$ con un promedio de éxito para esta prueba de $72\%$, es decir, un $8\%$ menos en comparación con el símil con todas las órdenes disponibles.

Este cambio se justifica principalmente por lo previamente comentado, al nosotros estar trabajando con las voluntades de compra y venta con mayor cercanía podemos decir que estamos un poco más cerca de "definir" la tendencia en el precio, y lógicamente ese cambio se ve más reflejado en comparativa a si tomamos todas las órdenes disponibles de cada _orderbook_, no por mucho pero si se logra distinguir esa pequeña diferenciación. 

### <font color= #6B6B6B> 4.2.2. Weighted Mid-Price </font>

Como sabemos el **Weigthed Mid-Price** tiene diferentes perspectivas y maneras de calcularse, sin embargo, por definición se esperaría que si bien las escalas o incluso las unidades entre las diversas metodologías de cálculo puede variar dependiendo el caso, en teoría o al menos la hipótesis que se plantea es que no debería existir un cambio radical en cuanto a comportamiento y el reflejo de mercado que cada una de ellas aporta.

Para poder corroborar dicha situación se utilizarán los mismos experimentos definidos para el modelo de **APT**. Este primer enfoque con el cual se propone calcular el **Weighted Mid-Price** se define matemáticamente a través de la siguiente fórmula:

$$\text{Weighted Mid-Price} = \bigg(\frac{\sum_{i=1}^{n} \text{bv}_{i}}{\sum_{i=1}^{n} \text{bv}_{i}+\text{av}_{i}} \bigg) \bigg(\frac{\text{bp} + \text{ap}}{2} \bigg)$$

donde:

- $\text{bv}: \text{bid volume}$
- $\text{av}: \text{ask volume}$
- $\text{ap}: \text{ask price}$
- $\text{bp}: \text{bid price}$

En otras palabras esta manera de calcular el **Weighted Mid-Price** básicamente consiste en multiplicar el inbalance del libro de órdenes y multiplicarlo por el **Simple Mid-Price**.

In [12]:
### Let's see now the tob experiments for the first way of calculation of weighted mid-price
print(f'''The min proportion where mid-price follows martingale process is {apt_tob['weighted_mid_price_a']['P. Exp 1'].min()}
and it happened on {apt_tob['weighted_mid_price_a']['P. Exp 1'].idxmin()}''')
print("")
print(f'''The mean proportion where the mid-price follows martingale process is:
{round(apt_tob['weighted_mid_price_a']['P. Exp 1'].mean(),4)*100}%''')

apt_tob['weighted_mid_price_a'].head()

The min proportion where mid-price follows martingale process is 0.65
and it happened on 2021-07-05 13:08:00

The mean proportion where the mid-price follows martingale process is:
66.92%


,Exp 1,Exp 2,P. Exp 1,P. Exp 2
Time,,,,
2021-07-05 13:06:00,6,3,0.67,0.33
2021-07-05 13:07:00,27,14,0.66,0.34
2021-07-05 13:08:00,26,14,0.65,0.35
2021-07-05 13:09:00,26,13,0.67,0.33
2021-07-05 13:10:00,27,14,0.66,0.34


Si comparamos el resultado obtenido con respecto al experimento realizado para todas las órdenes registradas vemos que existe una diferencia muy significativa ya que recordemos que para el caso del primero se contaba con un proporción promedio de cumplimiento del modelo de aproximadamente $0.02\%$, valor que si contrastamos con este nuevo vemos que existe una mejora del $64\%$.

Entonces se había comentado anteriormente que al nosotros estar trabajando con órdenes del _top of the book_ podríamos llegar a esperar un mayor cambio o movimiento en el precio de libro a libro, sin embargo, para el caso del **Weigthed Mid-Price** logramos percibir que mejora bastante en comparación al uso de todas las órdenes, ¿por qué?. Nuevamente para responder esta pregunta nos debemos remontar a la definición y estructura de cada _orderbook_, y recalcar que estamos trabajando con los "mejores" **Bid** y **Ask** respectivamente, y es esta la situación la que justamente se debe resaltar, porque si bien es más probable que haya una mayor diferencia en el precio de libro a libro, al estar tomando solo este punto donde existe la mejor voluntad entre compradores y vendedores, también podemos decir que en temas de volumen puede llegar a existir una mayor paridad en comparación a lo planteado con todas las órdenes.

### <font color= #6B6B6B> 4.2.3. Comparing with alternative Weighted Mid-Price </font>

Para esta segunda parte de la experimentación con el **Weighted Mid-Price** se tomó la siguiente definición matemática:

$$\text{Weighted Mid-Price} = \bigg(\frac{\text{bv}}{\sum_{i=1}^{n}\big(\text{bv}_{i}+\text{av}_{i}\big)} \bigg) \text{ap} + \bigg(\frac{\text{av}}{\sum_{i=1}^{n}\big(\text{bv}_{i}+\text{av}_{i}\big)} \bigg) \text{bp}$$

donde:

- $\text{bv}: \text{bid volume}$
- $\text{av}: \text{ask volume}$
- $\text{ap}: \text{ask price}$
- $\text{bp}: \text{bid price}$

La idea del desarrollo de esta manera alternativa de cálculo es demostrar si es que empirícamente existe una diferenciación importante entre los métodos de cálculo entre los **Weighted Mid-Prices**.

In [13]:
### Now let's compare it with the alternative way of calculating weighted mid-price
print(f'''The min proportion where mid-price follows martingale process is {apt_tob['weighted_mid_price_b']['P. Exp 1'].min()}
and it happened on {apt_tob['weighted_mid_price_b']['P. Exp 1'].idxmin()}''')
print("")
print(f'''The mean proportion where the mid-price follows martingale process is:
{round(apt_tob['weighted_mid_price_b']['P. Exp 1'].mean(),4)*100}%''')

apt_tob['weighted_mid_price_b'].head()

The min proportion where mid-price follows martingale process is 0.65
and it happened on 2021-07-05 13:08:00

The mean proportion where the mid-price follows martingale process is:
68.87%


,Exp 1,Exp 2,P. Exp 1,P. Exp 2
Time,,,,
2021-07-05 13:06:00,6,3,0.67,0.33
2021-07-05 13:07:00,27,14,0.66,0.34
2021-07-05 13:08:00,26,14,0.65,0.35
2021-07-05 13:09:00,26,13,0.67,0.33
2021-07-05 13:10:00,27,14,0.66,0.34


Comparado los resultados obtenidos a este punto con respecto a los del experimento pasado, podemos observar que el cambio realmente no es muy drástico en términos de cumplimiento del modelo de **APT**, en promedio el cambio solo se refleja en un $2\%$ aproximadamente. Con esto en consideración podemos aceptar la hipótesis planteada con anterioridad (al menos desde un punto de vista empírico); el cálculo del **Weighted Mid-Price** puede llegar a fluctuar en términos de unidades o escala, sin embargo en teoría al menos los dos métodos de cálculo propuestos deberían reflejar la misma información en términos de la microestructura del mercado analizado.

<br>

# <font color= #6B6B6B> 5. Roll Model Testing  </font>

<hr style="border:0.01in solid gray"> </hr>

Al igual que con el proceso de prueba que se maneja para el modelo del **APT**, para estudiar el comportamiento de este modelo de **Roll** también se estuvo trabajando con una serie de funciones que logren agilizar el proceso de análisis de los resultados.

Entonces, antes de partir directamente a dichos resultados la intención es dar un contexto respecto a la derivación y potencial de aplicación de este modelo. Para ello lo primero que se hace es partir de las siguientes ecuaciones, las cuales definen el precio de transacción esperado.

$$m_{t} = m_{t-1}+u_{t}$$
$$p_{t} = m_{t}+Cq_{t}$$

donde:

- $m_{t}: \text{Representa el precio eficiente en el tiempo presente (no observado)}$
- $m_{t-1}: \text{Representa el precio eficiente a un rezago en el tiempo presente (ya observado)}$
- $p_{t}: \text{Precio de transacción en el tiempo presente (ya observado)}$
- $C: \text{Costo de transacción impuesto por los "dealers"}$
- $u_{t}: \text{Representa una componente aleatoria de ruido} \sim \mathcal{N} \big(0, \sigma^{2} \big)$
- $q_{t}: \text{Variable binaria de dirección de transacción}$

En este sentido $q_{t}$ es un proceso binario con probabilidad de ocurrencia para una **venta** del $50\%$ y por ende para una **compra** del complemento $(50\%)$. Es importante mencionar que la aplicación del modelo de **Roll** se da para mercados no orientados por órdenes, es decir, la interacción entre los inversionistas es directa con un _market maker_, es por ello que el objetivo final del **Roll Model** es poder definir un coste de transacción (y por ende un _spread_) derivado del uso de estas estructuras de mercado. Es a partir de esta circunstancia que el concepto de diferenciales de precio es muy importante $(\Delta p_{t})$.

$$\Delta p_{t} = Cq_{t} - Cq_{t-1} + u_{t}$$
$$\Delta p_{t-1} = -Cq_{t-2} + Cq_{t-1} - u_{t}$$

Ahora ya con los diferenciales de precio definidos a cada punto del tiempo, es importante recordar que operador es el que nos puede ayudar a encontrar un patrón de reconocimiento entre cada elemento de la serie de tiempo, como sabemos tanto la **covarianza** como la **correlación** son conceptos estadísticos que interrelacionan el comportamiento entre $2$ y más variables aleatorias, es por ello que se definen estos para encontrar dicha relación y alcanzar a definir dicho patrón de comportamiento. Si recordamos la **covarianza** entre dos variables aleatorias se define como:

$$\text{Cov}\big(x,y \big) = E \big[x y \big]$$

Por lo tanto para términos de aplicación del problema a modelar tendríamos que:

$$\text{Cov}\big(\Delta p_{t-1} \Delta p_{t} \big) = E \big[\Delta p_{t-1} \Delta p_{t} \big]$$

Desarrollando los términos de la multiplicación interna obtenemos que:

$$E \big[\Delta p_{t-1} \Delta p_{t} \big] = E \big[-C^{2}q_{t-1}^{2} \big] + E \big[C^{2}q_{t-1}q_{t-2} \big] + E \big[Cq_{t-1}u_{t-1} \big] + E \big[C^{2}q_{t}q_{t-1} \big] - E \big[C^{2}q_{t}q_{t-2} \big] - E \big[Cq_{t}u_{t-1} \big] + E \big[Cq_{t-1}u_{t} \big] - E \big[Cq_{t-2}u_{t} \big] - E \big[u_{t}u_{t-1} \big]$$

Ahora ya con los valores esperados definidos es justamente a este punto donde comienzan a entrar e interacturar las grandes suposiciones sobre las cuales se cimenta el modelo de **Roll**.

- $E \big[q_{t-1}q_{t} \big] = 0: \text{Existe independencia entre la naturaleza de las órdenes}$
- $E \big[u_{t} \big] = 0: \text{Por definición en cuanto a la distribución de dicha componente} \longrightarrow u_{t} \sim \mathcal{N} \big(0, \sigma^{2} \big)$
- $E \big[u_{t}^{2} \big] = \sigma^{2}_{u}: \text{Por definición en cuanto a la distribución de dicha componente} \longrightarrow u_{t} \sim \mathcal{N} \big(0, \sigma^{2} \big)$
- $C: \text{Comisión desconocida pero se asume constante}$
- $E \big[q_{t-1}^{2} \big] = 1 \text{ y } E \big[q_{t}^{2} \big] = 1: \text{Siempre existirá una orden ya sea de compra o de venta}$

Con todos los supuestos planteados y partiendo del desarrollo para $\text{Cov}\big(\Delta p_{t-1} \Delta p_{t} \big)$, obtenemos finalmente la simplificación final del modelo.

$$\gamma = -C^{2}$$

donde:

- $\gamma = \text{Cov}\big(\Delta p_{t-1} \Delta p_{t} \big)$

Ya con la expresión final para el cálculo de coste por transacción para el _market maker_ (el cual por obvias razones se ve cobrado implicitamente para los inversionistas), podemos definir el _spread_ como $2C$, esto para poder tomar ambas operaciones de compra-venta que se sostienen en cada operación. Además de poder definir el _spread_ con la implementación de dicho modelo, tomando en cuenta las dinámicas de la microestructura podemos definir también un valor teórico tanto para **Bid** como para **Ask**:

$$\text{Bid} = m_{t} - C$$

$$\text{Ask} = m_{t} + C$$

En este sentido y conociendo más a detalle la finalidad y aplicaciones propias del modelo de **Roll** se propone el siguiente foco de análisis para poder contrastar su aplicación con datos de la vida real.

- **Contrastar _spread_ teórico contra real:** Definir como es que se da la evolución del _spread_ teórico y contrastarlo directamente con el _spread_ real observado en el libro de órdenes.


- **Contrastar _bid_ teórico contra real:** Definir como es que se da la evolución del _bif_ teórico y contrastarlo directamente con el _bid_ real observado en el libro de órdenes.


- **Contrastar _ask_ teórico contra real:** Definir como es que se da la evolución del _ask_ teórico y contrastarlo directamente con el _ask_ real observado en el libro de órdenes.


- **Validar la independencia entre la naturaleza de las órdenes:** Definir el espectro y la convergencia de probabilidad en términos frecuentistas para cada tipo de orden concretada en los trades públicos.

In [ ]:
### Let's calculate the metrics for roll model
warnings.filterwarnings("ignore")
pt_data = dt.pt_data
roll_model = fn.roll_model_check(ob_data, pt_data)
print(f'The keys for roll model testing functions are: {roll_model.keys()}')

## <font color= #6B6B6B> 5.1. Testing Theoretical Metrics </font>

Al igual que con el modelo de **APT** las vertientes de análisis para el modelo de **Roll** pueden ser bastante amplias, al final del día lo que se busca es probarlo en un contexto real, midiendo la efectividad del mismo y el propio cumplimiento de sus supuestos. Para este caso de estudio en particular lo que se propone es definir a $m_{t}$ como el **Simple Mid-Price** un métrica que como observamos con anterioridad no fluctúa tanto por ventanas de tiempo muy pequeñas.

La idea principal será definir el _spread_ teórico que esta asociado o implicito en la serie histórica de esos precios (concretamente en el diferencial de estos), para ver no solo que tan lejano o cercano se encuentra en relación al _spread_ real observado de los datos (**ask - bid**), también para poder notar su evolución en el tiempo y comprobar si realmente se puede considerar un valor constante.

### <font color= #6B6B6B> 5.1.2. Theoretical Spread </font>

In [ ]:
### Let'see the data frame with the definition of theoretical spread vs real
roll_model['spread_definition'].iloc[:,0:8].head()

In [ ]:
roll_model['spread_definition'].iloc[:,0:8]['theoretical_spread']

<br>

# <font color= #6B6B6B> 6. Results and Visual Resources  </font>

<hr style="border:0.01in solid gray"> </hr>

Sin duda alguna las visualizaciones forman una parte fundamental en el proceso de abstracción de la información, los apoyos gráficos no solo nos ayudan a ver el panorama desde una perspectiva general, también da la oportunidad de observar tendencias en el comportamiento de los procesos analizados. En este sentido la idea de este apartado es proveer de una serie de apoyos visuales que ayuden a consolidar los conceptos previamente mencionados en los apartados de pruebas en los modelos de alta frecuencia.

La idea principal para todas las gráficas será la de un análisis de serie de tiempo, se quiere observar la evolución y desempeño correspondiente a cada modelo.

## <font color= #6B6B6B> 6.1. APT Model Charts </font>

**Hablar de la función generadora y mostrar docstring**

### <font color= #6B6B6B> 6.1.1. Experiment Frequency (All orders)</font>

In [ ]:
### Let's see how stands the proportion of times where the price follows a martingale process (all orders)
vz.plot_stacked_bar(apt_all['simple_mid_price'], minutes=30)

**Comentario sobre grafica**

### <font color= #6B6B6B> 6.1.2. Experiment Frequency (top of the book orders)</font>

In [ ]:
### Let's see how stands the proportion of times where the price follows a martingale process (tob orders)
vz.plot_stacked_bar(apt_tob['simple_mid_price'], minutes=30)

**Comentario sobre grafica**

## <font color= #6B6B6B> 6.2. Roll Model Charts </font>

**Hablar de la función generadora y mostrar docstring**

### <font color= #6B6B6B> 6.2.1. Theoretical Spread Definition </font>

In [ ]:
### Let's see how theoretical spread stands againts real observed spread
vz.plot_teo_spread(roll_model['spread_definition'])['spread']

**Comentario sobre grafica**

In [ ]:
### Let's see how is the difference between spreads (theoretical vs observed)
vz.plot_teo_spread(roll_model['spread_definition'])['diff']

**Comentario sobre grafica**

### <font color= #6B6B6B> 6.2.2. Theoretical Metrics Definition </font>

La intención de este apartado de visualizaciones es poder observar como es que se definen las métricas (**bid** y **ask**) teóricas a través del tiempo en términos comparativos (tanto entre ellas como contrastando con los datos reales).

In [ ]:
### Let's see how all of our theoretical metrics are defined
vz.plot_teo_spread(roll_model['spread_definition'])['theo']

**Comentario sobre la grafica**

In [ ]:
### Let's see how the bid value behave
vz.plot_teo_spread(roll_model['spread_definition'])['bid']

**Comentario sobre la grafica**

In [ ]:
### Let's see how the ask value behave
vz.plot_teo_spread(roll_model['spread_definition'])['ask']

**Comentario sobre la grafica**

In [ ]:
### Let's see how all of our real observed metrics are defined
vz.plot_teo_spread(roll_model['spread_definition'])['real']

**Comentario sobre la grafica**

### <font color= #6B6B6B> 6.2.3. Independence Between Orders Type </font>

In [ ]:
### Let's see the probability evolution within orders type
vz.plot_prob_evo(roll_model['prob_evolution'])

**Comentario sobre grafica**

<br>

# <font color= #6B6B6B> 7. Conclusions  </font>

<hr style="border:0.01in solid gray"> </hr>

<br>

# <font color= #6B6B6B> 8. References </font>

<hr style="border:0.01in solid gray"> </hr>

[1] Munnoz, 2020. Python project template. https://github.com/iffranciscome/python-project. (2021).

<br>